# 9   Building Feature Based Grammars

이 장에서는 context free grammars를 확장해서  자질 기반 문법(feature based grammars)에서 언어적 패턴과 문법적 구문의 종류를 살펴보려고 합니다.  격일치, , subcategorization, 무한의존구문등의 영어구문의 특증을 좀 더 살펴볼 것입니다. 

##9.1   Grammatical Features

<img src= "http://www.yalescientific.org/wp-content/uploads/2012/02/features-chomsky.jpg"><BR>
                                     noam-chomsky

이 장에서는 규칙 기반 문법(rule-based grammars)에 의거해서 features와 그들의 values를 뽑는 과정을 살펴볼 것입니다. 이 모든 것이 미리 갖추어져 있는 사전의 간단한 예제로부터 시작해봅시다. 

In [52]:
#nltk.download()
import nltk
from __future__ import division
import nltk, re, pprint

In [53]:
kim = {'CAT': 'NP', 'ORTH': 'Kim', 'REF': 'k'}
chase = {'CAT': 'V', 'ORTH': 'chased', 'REL': 'chase'}

kim과 chase가 공유하고 있는 feature는 CAT, ORTH이다. 
그리고 semantically-oriented feature는 REF, REL이다. 
####참고: CAT (grammatical category), ORTH (orthography, i.e.,spelling), REF(referent), REL(relation) 

In [54]:
chase['AGT'] = 'sbj'
chase['PAT'] = 'obj'

####참고:agent는 주어( subject)의 역할, patient는 목적어(object)의 역할 

In [55]:
sent = "Kim chased Lee"
tokens = sent.split()
lee = {'CAT': 'NP', 'ORTH': 'Lee', 'REF': 'l'}
def lex2fs(word):
    for fs in [kim, lee, chase]:
        if fs['ORTH'] == word:
            return fs
subj, verb, obj = lex2fs(tokens[0]), lex2fs(tokens[1]), lex2fs(tokens[2])
verb['AGT'] = subj['REF']
verb['PAT'] = obj['REF']
for k in ['ORTH', 'REL', 'AGT', 'PAT']: # check featstruct of 'chase'
    print "%-5s => %s" % (k, verb[k])

ORTH  => chased
REL   => chase
AGT   => k
PAT   => l


다른 동사도 해보자! surprise의 경우: object의 출발점"source" (SRC) ,  경험자"experiencer" (EXP)역 

In [56]:
surprise = {'CAT': 'V', 'ORTH': 'surprised', 'REL': 'surprise',
            'SRC': 'sbj', 'EXP': 'obj'}

질문: agent, patient, source, experiencer는 뭔가요? 사실 이것보다 더 많습니다. 다음 세종 전자사건에서 나타나는 의미역 목록을 봅시다. 

###참고: 세종 전자사전의 의미역 목록

[1 ] 행위주(Agent)
[2 ] 경험주(Experience)
[3 ] 심리행위주(Mental Agent)
[4 ] 동반주(Companion)
[5 ] 대상(Theme)
[6 ] 장소(Location)
[7 ] 도착점(Goal)
[8 ] 결과상태(Final State)
[9 ] 출발점(Source)
[10] 도구(Instrument)
[11] 영향주(Effector)
[12] 기준치 (Criterion)
[13] 방향(Direction)
[14] 목적 (Purpose)
[15] 내용(Content)


출처: 박철수외(2005), 한국어 용언 사전 기술을 위한 의미역 설정의 기본 문제들


예문) 철수가(행위주) 학교에(도착점) 가다. 


예문출처:김병수외(2006), 부트스트래핑 알고리즘을 이용한 한국어 격조사의 의미역 결정, 한국컴퓨터종합학술대회 논문집 Vol. 33, No. 1(B)

 ##격일치 Syntactic Agreement 
 
 성, 수 격의 일치를 보이는 언어가 많은데 그 주에서 오늘은 영어에서의 수 일치를 살펴봅니다. 
 3인칭 단수(3rd person) 현재일 때 s가 붙네요. 

    단수         	         복수
1st per	I run       	we run
2nd per	you run     	you run
3rd per	he/she/it runs	they run
 

약간 더 복잡한 문장으로  context free grammar 격일치를 encode해봅시다.
*this dogs run 과  *these dog runs. 이런 문장이 나오면 안됩니다.

	
S -> NP_SG VP_SG

S -> NP_PL VP_PL

NP_SG -> Det_SG N_SG

NP_PL -> Det_PL N_PL

VP_SG -> V_SG

VP_PL -> V_PL


Det_SG -> 'this'

Det_PL -> 'these'

N_SG -> 'dog'

N_PL -> 'dogs'

V_SG -> 'runs'

V_PL -> 'run'

####참고: SG(singular),   PL(plural)

##1.2  특성과 제약 (Using Attributes and Constraints)

<img src="http://www.nltk.org/book_1ed/tree_images/ch09-tree-1.png" width="200"><BR>
<img src="http://www.nltk.org/book_1ed/tree_images/ch09-tree-2.png" width="200"><BR>
<img src="http://www.nltk.org/book_1ed/tree_images/ch09-tree-3.png" width="200"><BR>
<img src="http://www.nltk.org/book_1ed/tree_images/ch09-tree-3.png" width="200"><BR>

####참고: 
수식관계 :Determiner한정사(명사 앞에 붙는 the, some, my 등과 같은 말들),
수number: Singular단수,Plural 복수


subtrees를 표시하고 Fail 도 표시 가능:

<img src="http://www.nltk.org/book/tree_images/ch09-tree-5.png" width="200"><BR>
<img src="http://www.nltk.org/book/tree_images/ch09-tree-7.png" width="200"><BR>

<img src="http://www.nltk.org/book/tree_images/ch09-tree-9.png" width="400"><BR>

한정사(determiner)도 이와 같이 처리할 수 있다.
Det[NUM=sg] -> 'the' | 'some' | 'any'
Det[NUM=pl] -> 'the' | 'some' | 'any'
Det[NUM=?n] -> 'the' | 'some' | 'any'

In [57]:
tokens = 'Kim likes children'.split()
from nltk import load_parser #[1]
cp = load_parser('grammars/book_grammars/feat0.fcfg', trace=2)  #[2]
for tree in cp.parse(tokens):
    print(tree)

|.Kim .like.chil.|
Leaf Init Rule:
|[----]    .    .| [0:1] 'Kim'
|.    [----]    .| [1:2] 'likes'
|.    .    [----]| [2:3] 'children'
Feature Bottom Up Predict Combine Rule:
|[----]    .    .| [0:1] PropN[NUM='sg'] -> 'Kim' *
Feature Bottom Up Predict Combine Rule:
|[----]    .    .| [0:1] NP[NUM='sg'] -> PropN[NUM='sg'] *
Feature Bottom Up Predict Combine Rule:
|[---->    .    .| [0:1] S[] -> NP[NUM=?n] * VP[NUM=?n] {?n: u'sg'}
Feature Bottom Up Predict Combine Rule:
|.    [----]    .| [1:2] TV[NUM='sg', TENSE='pres'] -> 'likes' *
Feature Bottom Up Predict Combine Rule:
|.    [---->    .| [1:2] VP[NUM=?n, TENSE=?t] -> TV[NUM=?n, TENSE=?t] * NP[] {?n: u'sg', ?t: u'pres'}
Feature Bottom Up Predict Combine Rule:
|.    .    [----]| [2:3] N[NUM='pl'] -> 'children' *
Feature Bottom Up Predict Combine Rule:
|.    .    [----]| [2:3] NP[NUM='pl'] -> N[NUM='pl'] *
Feature Bottom Up Predict Combine Rule:
|.    .    [---->| [2:3] S[] -> NP[NUM=?n] * VP[NUM=?n] {?n: u'pl'}
Feature Single Edge Fun

##1.3 용어

<img src=http://www.nltk.org/images/avm1.png ="200"><BR>
Figure 1.3: Rendering a Feature Structure as an Attribute Value Matrix

###9.2   Processing Feature Structures

Feature structures:FeatStruct(),  Atomic feature values: strings or integers.

In [58]:
fs1 = nltk.FeatStruct(TENSE='past', NUM='sg')
print fs1

[ NUM   = 'sg'   ]
[ TENSE = 'past' ]


In [59]:
fs1 = nltk.FeatStruct(PER=3, NUM='pl', GND='fem')
print fs1['GND']
fs1['CASE'] = 'acc'

fem


복잡한 value에 대해서도 feature structures 정의할 수 있다. 

In [60]:
fs2 = nltk.FeatStruct(POS='N', AGR=fs1)
print fs2

[       [ CASE = 'acc' ] ]
[ AGR = [ GND  = 'fem' ] ]
[       [ NUM  = 'pl'  ] ]
[       [ PER  = 3     ] ]
[                        ]
[ POS = 'N'              ]


In [61]:
print fs2['AGR']

[ CASE = 'acc' ]
[ GND  = 'fem' ]
[ NUM  = 'pl'  ]
[ PER  = 3     ]


In [62]:
print fs2['AGR']['PER']

3


다른 방법으로는 bracketed string을 이용하여 feature=value가 쌍으로 할 수도 있다. 

In [63]:
 print(nltk.FeatStruct("[POS='N', AGR=[PER=3, NUM='pl', GND='fem']]"))

[       [ GND = 'fem' ] ]
[ AGR = [ NUM = 'pl'  ] ]
[       [ PER = 3     ] ]
[                       ]
[ POS = 'N'             ]


여기서 잠깐 쉬어가는 류 파크칼의 김씨 찾기 찾기

<img src="http://image.cine21.com/cine21/poster/2010/1206/M0010005__.JPG" width="300"><BR>

In [64]:
print nltk.FeatStruct(name='Lee', telno='01 27 86 42 96', age=33)

[ age   = 33               ]
[ name  = 'Lee'            ]
[ telno = '01 27 86 42 96' ]


<img src= "http://www.nltk.org/images/dag01.png" width="300"><BR>
<img src= "http://www.nltk.org/images/dag02.png" width="300"><BR>

<img src= "http://www.nltk.org/images/dag04.png" width="500"><BR>
<img src= "http://www.nltk.org/images/dag03.png" width="500"><BR>

In [65]:
print nltk.FeatStruct("""[NAME='Lee', ADDRESS=(1)[NUMBER=74, STREET='rue Pascal'],
                         SPOUSE=[NAME='Kim', ADDRESS->(1)]]""")

[ ADDRESS = (1) [ NUMBER = 74           ] ]
[               [ STREET = 'rue Pascal' ] ]
[                                         ]
[ NAME    = 'Lee'                         ]
[                                         ]
[ SPOUSE  = [ ADDRESS -> (1)  ]           ]
[           [ NAME    = 'Kim' ]           ]


In [66]:
print nltk.FeatStruct("[A='a', B=(1)[C='c'], D->(1), E->(1)]")

[ A = 'a'             ]
[                     ]
[ B = (1) [ C = 'c' ] ]
[                     ]
[ D -> (1)            ]
[ E -> (1)            ]


##2.1 통합과 단일화

a.[NUMBER = 74]

<img src="http://www.nltk.org/images/dag04-1.png" width="300"><BR>

b.[NUMBER = 74          ]
  [STREET = 'rue Pascal']

<img src="http://www.nltk.org/images/dag04-2.png" width="100"><BR>

c.[NUMBER = 74          ]
  [STREET = 'rue Pascal']
  [CITY = 'Paris'       ]

<img src="http://www.nltk.org/images/dag04-3.png" width="300"><BR>

unify() method를 사용해서 두 개의 feature 합침

In [67]:
fs1 = nltk.FeatStruct(NUMBER=74, STREET='rue Pascal')
fs2 = nltk.FeatStruct(CITY='Paris')
print fs1.unify(fs2)

[ CITY   = 'Paris'      ]
[ NUMBER = 74           ]
[ STREET = 'rue Pascal' ]


In [68]:
print fs2.unify(fs1)

[ CITY   = 'Paris'      ]
[ NUMBER = 74           ]
[ STREET = 'rue Pascal' ]


Unification between FS0 and FS1가 path를 나누어 쓰기 때문에 생기는 재미 있는 현상 

structure-sharin에서의 unification interacts

In [69]:
fs0 = nltk.FeatStruct("""[NAME=Lee,
                          ADDRESS=[NUMBER=74,
                                   STREET='rue Pascal'],
                          SPOUSE= [NAME=Kim,
                                   ADDRESS=[NUMBER=74,
                                            STREET='rue Pascal']]]""")
print fs0

[ ADDRESS = [ NUMBER = 74           ]               ]
[           [ STREET = 'rue Pascal' ]               ]
[                                                   ]
[ NAME    = 'Lee'                                   ]
[                                                   ]
[           [ ADDRESS = [ NUMBER = 74           ] ] ]
[ SPOUSE  = [           [ STREET = 'rue Pascal' ] ] ]
[           [                                     ] ]
[           [ NAME    = 'Kim'                     ] ]


In [70]:
fs1 = nltk.FeatStruct("[SPOUSE = [ADDRESS = [CITY = Paris]]]")
print fs1.unify(fs0)

[ ADDRESS = [ NUMBER = 74           ]               ]
[           [ STREET = 'rue Pascal' ]               ]
[                                                   ]
[ NAME    = 'Lee'                                   ]
[                                                   ]
[           [           [ CITY   = 'Paris'      ] ] ]
[           [ ADDRESS = [ NUMBER = 74           ] ] ]
[ SPOUSE  = [           [ STREET = 'rue Pascal' ] ] ]
[           [                                     ] ]
[           [ NAME    = 'Kim'                     ] ]


두 사람의 주소가 같다. 이 부분은 어떻게 처리하는 것이 좋을까? 

In [71]:
fs2 = nltk.FeatStruct("""[NAME=Lee, ADDRESS=(1)[NUMBER=74, STREET='rue Pascal'],
                          SPOUSE=[NAME=Kim, ADDRESS->(1)]]""")
print fs1.unify(fs2)

[               [ CITY   = 'Paris'      ] ]
[ ADDRESS = (1) [ NUMBER = 74           ] ]
[               [ STREET = 'rue Pascal' ] ]
[                                         ]
[ NAME    = 'Lee'                         ]
[                                         ]
[ SPOUSE  = [ ADDRESS -> (1)  ]           ]
[           [ NAME    = 'Kim' ]           ]


 updating말고 copy로 처리하기! 

In [72]:
fs1 = nltk.FeatStruct("[ADDRESS1=[NUMBER=74, STREET='rue Pascal']]")
fs2 = nltk.FeatStruct("[ADDRESS1=?x, ADDRESS2=?x]")
print fs2

[ ADDRESS1 = ?x ]
[ ADDRESS2 = ?x ]


In [73]:
print fs2.unify(fs1)

[ ADDRESS1 = (1) [ NUMBER = 74           ] ]
[                [ STREET = 'rue Pascal' ] ]
[                                          ]
[ ADDRESS2 -> (1)                          ]


##9.3   Extending a Feature based Grammar

In [74]:
##3,1 하위 범주화 Subcategorization

(27)VP -> IV
    VP -> TV NP

보충어 complement 
(29)		
SBar -> Comp S
Comp -> 'that'

<img src= "http://www.nltk.org/book/tree_images/ch09-tree-10.png"><BR>

(31)		
V[SUBCAT=<NP, NP, PP>]


(32)		
V[SUBCAT=<NP>]: phrasal level에서 구현해 보기 
예) Kim put the book on the table.

<img src= "http://www.nltk.org/book/tree_images/ch09-tree-11.png"><BR>

## 3.2   Heads Revisited

드디어 X-bar Syntax

<img src= "http://www.nltk.org/book/tree_images/ch09-tree-13.png"><BR>

<img src= "http://www.nltk.org/book/tree_images/ch09-tree-13.png"><BR>

<img src= "http://www.nltk.org/book/tree_images/ch09-tree-14.png"><BR>

<img src= "http://www.aistudy.co.kr/linguistics/images/%EC%B5%9C%EC%86%8C%EC%9D%B4%EB%A1%A0%EC%9D%98%20%EB%AC%B8%EB%B2%95%EC%B2%B4%EA%B3%84.gif"><BR>

<img src= "https://i.ytimg.com/vi/uSpmf1Od_tY/hqdefault.jpg"><BR>

3.3   Auxiliary Verbs and Inversion이나 명령문 의문문에서 주어 술어 도치도 모두 처리할 수 있다  
생략합니다. 들어 주셔서 감사합니다. 